In [7]:
import pygplates
import numpy as np  
import matplotlib.pyplot as plt
import pandas as pd

import sys
sys.path.append('/Users/simon/GIT/pttx/')

from reconstruct_atom_rasters import *
from reconstruction_classes import *
from raster_reconstruction_classes import *

%matplotlib inline
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
rotation_filename = '/Applications/GPlates-2.0.0/SampleData/FeatureCollections/Rotations/Matthews_etal_GPC_2016_410-0Ma_GK07.rot'
static_polygon_filename = '/Applications/GPlates-2.0.0/SampleData/FeatureCollections/ContinentalPolygons/Matthews_etal_GPC_2016_ContinentalPolygons.gpmlz'

M2016 = ReconstructionModel('M2016')
M2016.add_rotation_model(rotation_filename)
M2016.add_static_polygons(static_polygon_filename)


In [53]:
grid_sampling=0.5
plotting = False

raster_file = '/Users/simon/Data/GMTdata/hawaii2017/earth_relief_10m.grd'
ptopo = GplatesRaster(raster_file)

time_step = 10
reconstruction_times = np.arange(10,11,10)

for reconstruction_time in reconstruction_times:
    
    print reconstruction_time
    
    from_time = 0.
    to_time = reconstruction_time

    (reconstructed_point_lons,
     reconstructed_point_lats,
     reconstructed_point_zvals) = reconstruct_raster(ptopo, 
                                                     M2016.static_polygons, 
                                                     M2016.rotation_model,
                                                     from_time, to_time, 
                                                     grid_sampling=grid_sampling)

    if plotting:
        plt.figure(figsize=(20,9))
        plt.scatter(reconstructed_point_lons,reconstructed_point_lats,
                    c=reconstructed_point_zvals,s=6,vmin=0,vmax=5000)
        plt.colorbar()
        #plt.savefig('./images/UnreconstructedPaleoDEM_%sMa.png' % reconstruction_time)
        plt.show()
        plt.close()
        
    else:
        print len(reconstructed_point_lons)
        


10
Reconstruct static polygons...
Find static polygons...
Group by polygons...
Reverse reconstruct points...
Sample present-day grid...
122174


In [55]:
(grid_longitudes,
 grid_latitudes) = np.meshgrid(np.arange(-180.,180.0001,grid_sampling), 
                               np.arange(-90.,90.0001,grid_sampling)) 


from scipy.interpolate.interpnd import _ndim_coords_from_arrays
from scipy.spatial import cKDTree

def xyz2grd(point_lons,point_lats,point_zvals,grid_lons,grid_lats):
# https://stackoverflow.com/questions/30655749/how-to-set-a-maximum-distance-between-points-for-interpolation-when-using-scipy

    grid_sampling = np.abs(grid_lats[1]-grid_lats[0])
    xy = np.vstack((point_lons, point_lats)).T

    # Construct kd-tree, functionality copied from scipy.interpolate
    tree = cKDTree(xy)
    xi = _ndim_coords_from_arrays((grid_lons, grid_lats), ndim=xy.shape[1])
    dists, indexes = tree.query(xi)

    grid_interp = griddata(xy, point_zvals, 
                           (grid_lons, grid_lats), 
                           method='nearest')

    # Copy original result but mask missing values with NaNs
    result = grid_interp[:]
    result[dists > grid_sampling/2.] = np.nan
    
    return result

def WriteCDFGrid(filename,x,y,z):
    ds = xr.DataArray(z, coords=[('y',y), ('x',x)], name='z')
    ds.to_netcdf(filename, format='NETCDF4')
    
    
res = xyz2grd(reconstructed_point_lons,reconstructed_point_lats,reconstructed_point_zvals,
              grid_longitudes,grid_latitudes)
plt.imshow(res,origin='lower')
